In [17]:
import pandas as pd
from datetime import datetime, timedelta

#load dataset from database inaportnet and scraping inaportnet
#load dataset from database inaportnet
df1 = pd.read_excel("D:\RKBM\RKBM.IDSRI.2022-01-01.xlsx")
df2 = pd.read_excel("D:\RKBM\RKBM.IDSRI.2022-02-01.xlsx")
df3 = pd.read_excel("D:\RKBM\RKBM.IDSRI.2022-03-01.xlsx")
df4 = pd.read_excel("D:\RKBM\RKBM.IDSRI.2022-04-01.xlsx")
df5 = pd.read_excel("D:\RKBM\RKBM.IDSRI.2022-05-01.xlsx")
df6 = pd.read_excel("D:\RKBM\RKBM.IDSRI.2022-06-01.xlsx")
df7 = pd.read_excel("D:\RKBM\RKBM.IDSRI.2022-07-01.xlsx")
df8 = pd.read_excel("D:\RKBM\RKBM.IDSRI.2022-08-01.xlsx")

#append dataset database inaportnet
ina_rkbm_database = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8])

#load dataset RKBM scrape
ina_rkbm_scrape = pd.read_excel("D:\RKBM\RKBMScrape.xlsx")

#join dataset inaportnet database and scraping
vlookup = pd.merge(ina_rkbm_scrape, ina_rkbm_database, left_on=['nomor produk'], right_on=['NOMOR RKBM'], how='inner')

#change waktu permohonan into Local Time
vlookup['waktu permohonan LT'] = vlookup['waktu permohonan']+timedelta(hours=8)

#calculate time difference
vlookup['selisih waktu'] = vlookup['waktu respon'] - vlookup['waktu permohonan LT']

#mencari bug atau tidak
sort = vlookup.sort_values('selisih waktu', ascending = False)
vlookup_tanpabug = vlookup[vlookup['selisih waktu']>timedelta(seconds=0)]
vlookup_tanpabug.sort_values('selisih waktu', ascending = True).head()

#make criteria in time difference
def sel(vlookup_tanpabug):
    if vlookup_tanpabug['selisih waktu'] < timedelta(seconds=31):
        return "Kurang dari 30 detik"
    elif vlookup_tanpabug['selisih waktu'] < timedelta(minutes=30):
        return "Antara 30 detik - 30 menit"
    elif vlookup_tanpabug['selisih waktu'] < timedelta(hours=1):
        return "Antara 30 - 60 menit"
    elif vlookup_tanpabug['selisih waktu'] < timedelta(hours=6):
        return "Antara 1 - 6 jam"
    elif vlookup_tanpabug['selisih waktu'] < timedelta(hours=12):
        return "Antara 6 - 12 jam"
    elif vlookup_tanpabug['selisih waktu'] < timedelta(days=1):
        return "Antara 12 - 24 jam"
    else:
        return "Lebih dari 24 jam"

vlookup_tanpabug['pembagian selisih waktu'] = vlookup_tanpabug.apply(sel, axis=1)
sort_data = vlookup_tanpabug.sort_values("selisih waktu", axis = 0, ascending = True)

#grouping pembagian selisih waktu
waktu_pelayanan = vlookup_tanpabug.groupby('pembagian selisih waktu')['pembagian selisih waktu'].count().reset_index(name='jumlah')
waktu_pelayanan['persentase'] = waktu_pelayanan['jumlah']/len(vlookup_tanpabug)*100
waktu_pelayanan.head(7)

#indentasi penomoran waktu pelayanan
def nomor_waktu(waktu_pelayanan):
    if waktu_pelayanan['pembagian selisih waktu'] == 'Kurang dari 30 detik':
        return 1
    elif waktu_pelayanan['pembagian selisih waktu'] == 'Antara 30 detik - 30 menit':
        return 2
    elif waktu_pelayanan['pembagian selisih waktu'] == 'Antara 30 - 60 menit':
        return 3
    elif waktu_pelayanan['pembagian selisih waktu'] == 'Antara 1 - 6 jam':
        return 4
    elif waktu_pelayanan['pembagian selisih waktu'] == 'Antara 6 - 12 jam':
        return 5
    elif waktu_pelayanan['pembagian selisih waktu'] == 'Antara 12 - 24 jam':
        return 6
    else:
        return 7

#grouping waktu pelayanan
waktu_pelayanan['Nomor'] = waktu_pelayanan.apply(nomor_waktu, axis=1)
waktu_pelayanan = waktu_pelayanan.sort_values("Nomor", axis = 0, ascending = True)

#membuat table hari, bulan, waktu permohonan
#divide waktu permohonan into day, month, time
month_mapping={
    1: 'January', 
    2: 'February', 
    3: 'March', 
    4: 'April', 
    5: 'May',
    6: 'June', 
    7: 'July',
    8: 'August'
} 
dw_mapping={
    0: 'Monday', 
    1: 'Tuesday', 
    2: 'Wednesday', 
    3: 'Thursday', 
    4: 'Friday',
    5: 'Saturday', 
    6: 'Sunday'
} 

vlookup_tanpabug['bulan permohonan'] = vlookup_tanpabug['waktu permohonan LT'].dt.month.map(month_mapping)
vlookup_tanpabug['hari permohonan'] = vlookup_tanpabug['waktu permohonan LT'].dt.dayofweek.map(dw_mapping)

#grouping pembagian bulan permohonan
bulan_permohonan = vlookup_tanpabug.groupby('bulan permohonan')['bulan permohonan'].count().reset_index(name='jumlah')
bulan_permohonan['persentase'] = bulan_permohonan['jumlah']/len(vlookup)*100

def nomor(bulan_permohonan):
    if bulan_permohonan['bulan permohonan'] == 'January':
        return 1
    elif bulan_permohonan['bulan permohonan'] == 'February':
        return 2
    elif bulan_permohonan['bulan permohonan'] == 'March':
        return 3
    elif bulan_permohonan['bulan permohonan'] == 'April':
        return 4
    elif bulan_permohonan['bulan permohonan'] == 'May':
        return 5
    elif bulan_permohonan['bulan permohonan'] == 'June':
        return 6
    elif bulan_permohonan['bulan permohonan'] == 'July':
        return 7
    else:
        return 8

bulan_permohonan['nomor'] = bulan_permohonan.apply(nomor, axis=1)
bulan_permohonan = bulan_permohonan.sort_values("nomor", axis = 0, ascending = True)
bulan_permohonan

#grouping pembagian bulan permohonan
def nomor_hari(hari_permohonan):
    if hari_permohonan['hari permohonan'] == 'Monday':
        return 1
    elif hari_permohonan['hari permohonan'] == 'Tuesday':
        return 2
    elif hari_permohonan['hari permohonan'] == 'Wednesday':
        return 3
    elif hari_permohonan['hari permohonan'] == 'Thursday':
        return 4
    elif hari_permohonan['hari permohonan'] == 'Saturday':
        return 5
    elif hari_permohonan['hari permohonan'] == 'Sunday':
        return 6
    else:
        return 7
    
hari_permohonan = vlookup_tanpabug.groupby('hari permohonan')['hari permohonan'].count().reset_index(name='jumlah')
hari_permohonan['Nomor'] = hari_permohonan.apply(nomor_hari, axis=1)
hari_permohonan = hari_permohonan.sort_values("Nomor", axis = 0, ascending = True)
hari_permohonan

#pembagian kriteria berdasarkan waktu permohonan
batas_waktu1 = pd.to_datetime('06:00:00', format='%H:%M:%S')
batas_waktu2 = pd.to_datetime('12:00:00', format='%H:%M:%S')
batas_waktu3 = pd.to_datetime('18:00:00', format='%H:%M:%S')

#kriteria jam permohonan
def jam(vlookup_tanpabug):
    if vlookup_tanpabug['jam permohonan'] < batas_waktu1:
        return "Jam 00.00 s.d. 06.00"
    elif vlookup_tanpabug['jam permohonan'] < batas_waktu2:
        return "Jam 06.00 s.d. 12.00"
    elif vlookup_tanpabug['jam permohonan'] < batas_waktu3:
        return "Jam 12.00 s.d. 18.00"
    else:
        return "Jam 18.00 s.d. 24.00"

vlookup_tanpabug['jam permohonan'] = vlookup_tanpabug['waktu permohonan LT'].dt.time
vlookup_tanpabug['jam permohonan'] = pd.to_datetime(vlookup_tanpabug['jam permohonan'], format='%H:%M:%S')
vlookup_tanpabug['pembagian jam permohonan'] = vlookup_tanpabug.apply(jam, axis=1)

#grouping pembagian jam permohonan
waktu_permohonan = vlookup_tanpabug.groupby('pembagian jam permohonan')['pembagian jam permohonan'].count().reset_index(name='jumlah')
waktu_permohonan['persentase'] = waktu_permohonan['jumlah']/len(vlookup_tanpabug)*100

<ipython-input-17-9213beef26ad>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vlookup_tanpabug['pembagian selisih waktu'] = vlookup_tanpabug.apply(sel, axis=1)
<ipython-input-17-9213beef26ad>:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vlookup_tanpabug['bulan permohonan'] = vlookup_tanpabug['waktu permohonan LT'].dt.month.map(month_mapping)
<ipython-input-17-9213beef26ad>:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [18]:
waktu_pelayanan

,pembagian selisih waktu,jumlah,persentase,Nomor
5,Kurang dari 30 detik,461,1.885558,1
3,Antara 30 detik - 30 menit,10081,41.232770,2
2,Antara 30 - 60 menit,3928,16.066097,3
0,Antara 1 - 6 jam,8465,34.623093,4
4,Antara 6 - 12 jam,786,3.214855,5
1,Antara 12 - 24 jam,573,2.343654,6
6,Lebih dari 24 jam,155,0.633973,7


In [19]:
bulan_permohonan

,bulan permohonan,jumlah,persentase,nomor
3,January,649,2.621904,1
2,February,2664,10.762332,2
6,March,4053,16.373773,3
0,April,4002,16.167737,4
7,May,3815,15.412273,5
5,June,4257,17.197915,6
4,July,3195,12.907526,7
1,August,1814,7.328405,8


In [20]:
hari_permohonan

,hari permohonan,jumlah,Nomor
1,Monday,4332,1
5,Tuesday,4197,2
6,Wednesday,4027,3
4,Thursday,3996,4
2,Saturday,1825,5
3,Sunday,1259,6
0,Friday,4813,7


In [21]:
waktu_permohonan

,pembagian jam permohonan,jumlah,persentase
0,Jam 00.00 s.d. 06.00,328,1.341568
1,Jam 06.00 s.d. 12.00,5778,23.632868
2,Jam 12.00 s.d. 18.00,14358,58.726328
3,Jam 18.00 s.d. 24.00,3985,16.299235
